In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
houses = pd.read_csv('data/cal_housing_clean.csv')

In [ ]:
houses.head()

In [ ]:
houses = houses.apply(lambda x: (x - x.min()) / (x.max()-x.min() ))

In [ ]:
housingMedianAge = tf.feature_column.numeric_column('housingMedianAge')
totalRooms = tf.feature_column.numeric_column('totalRooms')
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')
medianHouseValue = tf.feature_column.numeric_column('medianHouseValue')

feat_cols = [housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome]

In [ ]:
x_data = houses.drop('medianHouseValue',axis=1)
label = houses['medianHouseValue']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data,label,test_size = 0.3)

In [ ]:
# Training process

In [ ]:
train_inputFunc = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,batch_size=20,num_epochs=1000,shuffle=True)

In [ ]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

In [ ]:
dnn_model.train(input_fn=train_inputFunc,steps=10000)

In [ ]:
# Prediction process

In [ ]:
pred_inputFunc = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=20,num_epochs=1,shuffle=False)

In [ ]:
pred_gen = dnn_model.predict(pred_inputFunc)

In [ ]:
predictions = list(pred_gen)

In [ ]:
final_preds =[]
for preds in predictions:
    final_preds.append(preds['predictions'])

In [ ]:
mean_squared_error(y_test,final_preds)**0.5

In [ ]:
houses.describe().transpose()